In [1]:
import pandas as pd
import ast

In [2]:
dataset_name = ['ADHD', 'Blind', 'Disability']
df_list = []
for d in dataset_name:
    df_list.append(pd.read_csv(f'datasets/2nd_filtering/{d}_relevance.csv', index_col=0))


In [3]:
for i, d in enumerate(dataset_name):
    df = df_list[i]
    df['token_count'] = df['anonymized_body_lemmatized'].apply(lambda x: len(ast.literal_eval(x)))
    df['token_type_count'] = df['anonymized_body_lemmatized'].apply(lambda x: len(set(ast.literal_eval(x))))
    print(f'=====dataset:{d}======')
    print(f"Posts:{df.shape[0]}")
    print(f"Tokens:{df['token_count'].sum()}")
    print(f"Types:{df['token_type_count'].sum()}")
    print(f"TTR:{(df['token_type_count'].sum()/df['token_count'].sum()*1000).round(2)}")
    print(f"Avg. of tokens:{df['token_count'].mean().round(2)}")
    print(f"Avg. of types:{df['token_type_count'].mean().round(2)}")
    sentiment = df['post_sentiment_label'].apply(lambda x: 1 if x == 'negative' else 0)
    print(f"Sentiment ratio:{(sentiment.value_counts()[1]/sentiment.value_counts()[0]).round(2)}")
    print(f"Relevance ratio:{(df['Relevance'].value_counts()[1]/df['Relevance'].value_counts()[0]).round(2)}")

=====dataset:ADHD======
Posts:56197
Tokens:10812473
Types:5613877
TTR:519.2
Avg. of tokens:192.4
Avg. of types:99.9
Sentiment ratio:1.06
Relevance ratio:13.64
=====dataset:Blind======
Posts:1008
Tokens:214930
Types:108452
TTR:504.59
Avg. of tokens:213.22
Avg. of types:107.59
Sentiment ratio:0.74
Relevance ratio:10.33
=====dataset:Disability======
Posts:2434
Tokens:527366
Types:264690
TTR:501.91
Avg. of tokens:216.67
Avg. of types:108.75
Sentiment ratio:1.51
Relevance ratio:14.6


In [20]:
predictions = pd.read_csv('all_predictions.csv', index_col =0)
scores = pd.read_csv('all_scores.csv', index_col =0)


In [31]:
summary = []
for m in scores['model'].unique():
    score = scores[(scores['dataset'] == 'Disability') & (scores['model'] == m)]
    summary.append({'model': m, 'acc': score['acc'].mean(), 'prec': score['prec'].mean(), 'rec': score['rec'].mean(), 'f1-score': score['f1-score'].mean()})
pd.DataFrame(summary)

In [32]:
pd.DataFrame(summary)

,model,acc,prec,rec,f1-score
0,llama2_7b_pred,0.585,0.547472,1.00,0.707261
1,llama2_13b_pred,0.850,0.888889,0.80,0.842105
2,TimeLMs_label,0.500,0.500000,1.00,0.666667
3,hatexplain_label,0.500,0.500000,1.00,0.666667
4,llama2_7b_FL_pred,0.730,0.652871,0.99,0.786254
5,llama2_13b_FL_pred,0.665,1.000000,0.33,0.494505
